##torch.nn.functional.grid_sample 의 작동 방식

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

grid_sample에는 source가 될 tensor [B, C, H, W] 와 warping을 위해 필요한 grid (어떻게 source image에서 가져올 것인지에 대한 값) 이 필요하다  
source tensor를 다음과 같이 정의해보자

In [104]:
#source tensor -> temp_input
temp_input = torch.tensor([[1, 2, 3],[4, 5, 6], [7, 8, 9]])
temp_input = temp_input.reshape(1, 1, 3, 3)
print(temp_input)
print(temp_input.shape)

tensor([[[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]])
torch.Size([1, 1, 3, 3])


grid는 [B, H, W, 2] 의 shape을 갖게 되고, 2는 x, y를 나타낸다 [B, H, W, xy]  
grid의 값은 -1 ~ 1의 값을 가질 수 있고, -1인 경우 tensor의 x는 left, y는 top, 1인 경우 x는 right,  y는 bottom을 의미한다

In [92]:
grid = torch.ones((1,3,3,2))

In [93]:
grid

tensor([[[[1., 1.],
          [1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.],
          [1., 1.]]]])

현재 모든 grid의 1이므로 값이 x는 가장 right, y는 가장 bottom 에 있는 값을 가져오라는 의미를 가진 grid가 생성되었다

In [80]:
temp_input

tensor([[[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]])

temp_input에서의 가장 right bottom에 있는 값은 9이다

In [94]:
warp = F.grid_sample(temp_input.float(), grid, align_corners=True)

In [95]:
warp

tensor([[[[9., 9., 9.],
          [9., 9., 9.],
          [9., 9., 9.]]]])

warpping된 값이 모두 9의 값으로 채워진 것을 알 수 있다  
반대로 모두 -1의 값을 갖는다면, 각 자리에서 모두 source의 left top의 값을 가져오게 되니 모든 값이 1의 값을 갖는다

In [96]:
grid[...] = -1
warp = F.grid_sample(temp_input.float(), grid, align_corners=True)
print(warp)

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])


## n차원의 경우  
n차원의 경우, 사진을 생각해보면 3 channel이 되고 각 x,y자리마다 3개의 값을 가지고 있게 된다  
이 3개의 값이 하나가 된다고 생각하고 x, y 만 고려해서 해당 값들이 모두 자리를 찾아간다  
다음의 3D tensor를 만들어 보았다  
<img src="../images/etc/temp_tensor.png" width="500" height="300">  
3Channel Tensor이고, 각 channel의 같은 x,y에는 같은 값이 들어있다  


In [97]:
temp_input = torch.tensor([[1, 2, 3],[4, 5, 6], [7, 8, 9]])
temp_input = temp_input.reshape(1, 1, 3, 3).repeat(1,3,1,1)

In [98]:
temp_input

tensor([[[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]],

         [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]],

         [[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]])

In [101]:
grid = torch.zeros((1,3,3,2))
#0,0자리에 값을 가져올 위치의 x를 가장 Right의 위치로 설정
grid[:,0,0,0] = 1
#0,0자리에 값을 가져올 위치의 y를 가장 Bottom의 위치로 설정
grid[:,0,0,1] = 1
grid

tensor([[[[1., 1.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]]])

0,0에 자리하게 될 값은 1,1에 위치한 즉, 가장 right bottom에 위치한 9,9,9의 값을 가져오게 된다  
나머지는 0,0에 에 위치한, source image에서 정중앙에 위치한 5의 값을 가져오게 된다  
channel이 늘어나도 해당 자리의 값들을 쏙쏙 뽑아다가 자리에 둔다고 생각하면 됨!  

In [102]:
warp = F.grid_sample(temp_input.float(), grid, align_corners=True)

In [103]:
warp

tensor([[[[9., 5., 5.],
          [5., 5., 5.],
          [5., 5., 5.]],

         [[9., 5., 5.],
          [5., 5., 5.],
          [5., 5., 5.]],

         [[9., 5., 5.],
          [5., 5., 5.],
          [5., 5., 5.]]]])

<img src="../images/etc/warp_tensor.png" width="800" height="300">  
